# Implement AutoEncoder based on frequency domain features

In [1]:
import pandas as pd
from utils import Packet, Flow

def get_flows(df: pd.DataFrame, key_type: str = "default") -> dict:
    mp = dict()
    for idx in range(len(df)): # simulate the process of packet processing
        row = df.iloc[idx]
        pkt = Packet(
            src_ip=row["src_ip"],
            dst_ip=row["dst_ip"],
            src_port=row["src_port"],
            dst_port=row["dst_port"],
            protocol=row["protocol"],
            proto_code=row["proto_code"],
            pkt_length=row["pkt_length"],
            timestamp=row["timestamp"],
            ttl=row["ttl"],
            tcp_window=row["tcp_window"],
            tcp_dataoffset=row["tcp_dataoffset"],
            udp_length=row["udp_length"],
            label=row["label"],
        )
        key = pkt.key(type=key_type)
        if key not in mp:
            mp[key] = Flow()
        mp[key].add_packet(pkt)
    return mp

In [6]:
import torch
from config import whisper_config

def transform(mp: dict, feature_type: str = "whisper", 
              data_type: str = "train", test_data_aug: bool = True):
    packet_data, flow_data = [], []
    packet_labels, flow_labels = [], []
    for key, flow in mp.items():
        vec = flow.vector(feature_type=feature_type)
        if feature_type == "whisper":
            if len(vec) <= (whisper_config["n_fft"] // 2):
                # packet level features
                # vec = flow.packet_vector(agg_type="mean") + flow.packet_vector(agg_type="std") \
                #     + flow.packet_vector(agg_type="max") + flow.packet_vector(agg_type="min")
                # packet_data.append(vec)
                # packet_labels.append(flow.label)

                # implement fft on short flows
                ten = torch.tensor(vec)
                ten_fft = torch.fft.fft(ten, n=(whisper_config["n_fft"] // 2)+1)
                ten_power = torch.pow(ten_fft.real, 2) + torch.pow(ten_fft.imag, 2)
                ten_res = (ten_power.squeeze()+1).log2()
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                if data_type == "test" and test_data_aug:
                    # data shape for test data augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    packet_data.append([ten_res.tolist()])
                else:
                    # data shape for no data augmentation: (n_flow, floor(n_fft/2)+1)
                    packet_data.append(ten_res.tolist())
                packet_labels.append(flow.label)
                
            else:
                # flow level featrues
                ten = torch.tensor(vec)
                # stft requirement: input_size > (n_fft // 2)
                # default return shape: (floor(n_fft/2)+1, n_frame, 2)
                ten_fft = torch.stft(ten, whisper_config["n_fft"])
                ten_power = torch.pow(ten_fft[:,:,0], 2) + torch.pow(ten_fft[:,:,1], 2)
                ten_res = ((ten_power.squeeze()+1).log2()).permute(1,0)
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                # ten_res shape: (n_frame, floor(n_fft/2)+1)
                if data_type == "train":
                    if (ten_res.size(0) > whisper_config["mean_win_train"]):
                        for _ in range(whisper_config["num_train_sample"]):
                            start_idx = torch.randint(0, ten_res.size(0)
                                        - whisper_config["mean_win_train"], (1,)).item()
                            ten_tmp = ten_res[start_idx:start_idx+whisper_config["mean_win_train"],:].mean(dim=0)
                            flow_data.append(ten_tmp.tolist())
                            flow_labels.append(flow.label)
                    else:
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        flow_labels.append(flow.label)
                else: # for test
                    if test_data_aug:
                        tmp_data = []
                        if (ten_res.size(0) > whisper_config["mean_win_test"]):
                            # data augmentation for kmeans on flows with length > mean_win_test
                            for idx in range(0, ten_res.size(0) - whisper_config["mean_win_test"], 
                                            whisper_config["mean_win_test"]):
                                ten_tmp = ten_res[idx:idx+whisper_config["mean_win_test"],:].mean(dim=0)
                                tmp_data.append(ten_tmp.tolist())
                        else:
                            # no data augmentation for kmeans on flows with length < mean_win_test
                            tmp_data.append(ten_res.mean(dim=0).tolist())
                        flow_data.append(tmp_data)
                        # data shape for augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    else: # for other detection methods
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        # data shape for no augmentation: (n_flow, floor(n_fft/2)+1)
                    flow_labels.append(flow.label)
        elif feature_type == "encoding":
            # directly use the whisper encoding vector
            pass
        else: # for other feature types
            pass
    return packet_data, packet_labels, flow_data, flow_labels

In [47]:
import torch.nn as nn

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Sigmoid())
        # self.decoder = nn.Sequential(
        #     nn.Linear(hidden_dim, input_dim),
        #     nn.Sigmoid())
        self.decoder = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        super(Dataset, self).__init__()
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

## Training

In [21]:
import pandas as pd

USE_SHORT_FLOW = True

train_benign_filename = "dataset/benign_small.csv"
train_df = pd.read_csv(train_benign_filename)
train_df["label"] = 1
train_packet_data, train_packet_labels, train_flow_data, train_flow_labels = transform(get_flows(train_df))
train_data = train_flow_data if not USE_SHORT_FLOW else train_flow_data + train_packet_data
train_labels = train_flow_labels if not USE_SHORT_FLOW else train_flow_labels + train_packet_labels

In [48]:
import json
import os
import torch

def train_ae(train_data, train_labels, save_dir,
            model, criterion, optimizer, device, 
            batch_size=32, num_epochs=200):
    train_dataset = Dataset(train_data, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    loss_list = []
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for data, labels in train_loader:
            # data = torch.sigmoid(data.to(device))
            data = data.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, data)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    os.makedirs(save_dir, exist_ok=True)
    model_save_path = os.path.join(save_dir, "model.pt")
    torch.save(model.state_dict(), model_save_path)
    loss_save_path = os.path.join(save_dir, "train_loss.json")
    with open(loss_save_path, "w") as f:
        json.dump(loss_list, f)

In [52]:
from config import whisper_config
import torch
import os

input_dim = whisper_config["n_fft"] // 2 + 1
hidden_dim = int(input_dim * 0.75)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoEncoder(input_dim, hidden_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

save_dir = os.path.join("model", "whisper", "autoencoder", os.path.basename(train_benign_filename))
train_ae(torch.tensor(train_data), torch.tensor(train_labels),
        save_dir, model, criterion, optimizer, device, num_epochs=50)

Epoch 1/50, Loss: 31.2048
Epoch 2/50, Loss: 4.9135
Epoch 3/50, Loss: 5.0716
Epoch 4/50, Loss: 4.2535
Epoch 5/50, Loss: 4.3831
Epoch 6/50, Loss: 4.9766
Epoch 7/50, Loss: 5.2624
Epoch 8/50, Loss: 7.5395
Epoch 9/50, Loss: 4.6199
Epoch 10/50, Loss: 4.5326
Epoch 11/50, Loss: 6.1677
Epoch 12/50, Loss: 4.3662
Epoch 13/50, Loss: 4.5696
Epoch 14/50, Loss: 5.2829
Epoch 15/50, Loss: 4.3696
Epoch 16/50, Loss: 4.4715
Epoch 17/50, Loss: 4.5239
Epoch 18/50, Loss: 5.1906
Epoch 19/50, Loss: 4.2480
Epoch 20/50, Loss: 4.8013
Epoch 21/50, Loss: 5.0120
Epoch 22/50, Loss: 4.2789
Epoch 23/50, Loss: 5.3500
Epoch 24/50, Loss: 4.5157
Epoch 25/50, Loss: 4.2984
Epoch 26/50, Loss: 4.9363
Epoch 27/50, Loss: 5.6220
Epoch 28/50, Loss: 4.2590
Epoch 29/50, Loss: 4.8429
Epoch 30/50, Loss: 4.3347
Epoch 31/50, Loss: 4.3436
Epoch 32/50, Loss: 4.1997
Epoch 33/50, Loss: 4.0871
Epoch 34/50, Loss: 4.9693
Epoch 35/50, Loss: 5.1477
Epoch 36/50, Loss: 5.6590
Epoch 37/50, Loss: 4.8395
Epoch 38/50, Loss: 4.6746
Epoch 39/50, Loss: 4

In [27]:
import os

benign_filenames = [os.path.join("train_set", "benign" + str(i) + ".csv") 
                    for i in range(1, 3)] + ["dataset_lite/mirai-benign.csv"]
attack_filenames = [os.path.join("dataset_lite", x+".csv") for x in 
                    ["BruteForce-Web", "BruteForce-XSS", "infiltration", 
                    "osscan", "SQL_Injection", "ssldosA10only", "mirai-attack"]]

In [50]:
from sklearn.metrics import accuracy_score

def test_ae(test_data, test_labels, model, device, 
            threshold, scale=5, test_data_aug=False):
    model.eval()
    preds = []
    with torch.no_grad():
        for val in test_data:
            # data = torch.sigmoid(torch.tensor(val).to(device))
            data = torch.tensor(val).to(device)
            outputs = model(data)
            loss = criterion(outputs, data)
            if not test_data_aug:
                preds.append(-1 if loss.item() > threshold * scale else 1)
            else:
                preds.append(-1 if loss.max().item() > threshold * scale else 1)
    return accuracy_score(test_labels, preds)

In [55]:
import torch
import json
from config import whisper_config

USE_DATA_AUG = True
detect_type = "autoencoder" if USE_DATA_AUG else "autoencoder-no-aug"
accuracy_dict = {}

imput_dim = whisper_config["n_fft"] // 2 + 1
hidden_dim = int(input_dim * 0.75)
model = AutoEncoder(input_dim, hidden_dim)
model.load_state_dict(torch.load(os.path.join(save_dir, "model.pt")))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with open(os.path.join(save_dir, "train_loss.json"), "r") as f:
    loss_list = json.load(f)
threshold = torch.tensor(loss_list).mean().item()

for test_benign_filename in benign_filenames:
    test_df = pd.read_csv(test_benign_filename)
    test_df["label"] = 1
    test_packet_data, test_packet_lables, test_flow_data, test_flow_labels \
    = transform(get_flows(test_df), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_lables
    acc = test_ae(test_data, test_labels, model, device, threshold, scale=1.5)
    print(f"accuracy of {test_benign_filename}: {acc}")
    accuracy_dict[test_benign_filename] = acc

for test_attack_filename in attack_filenames:
    test_df = pd.read_csv(test_attack_filename)
    test_df["label"] = -1
    test_packet_data, test_packet_lables, test_flow_data, test_flow_labels \
    = transform(get_flows(test_df), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_lables
    acc = test_ae(test_data, test_labels, model, device, threshold, scale=1.5)
    print(f"accuracy of {test_attack_filename}: {acc}")
    accuracy_dict[test_attack_filename] = acc

accuracy_base_name = "flow-accuracy.json" if not USE_SHORT_FLOW else "all-accuracy.json"
accuracy_save_path = os.path.join("result", "whisper", detect_type, os.path.basename(train_benign_filename), accuracy_base_name)
os.makedirs(os.path.dirname(accuracy_save_path), exist_ok=True)
with open(accuracy_save_path, "w") as f:
    json.dump(accuracy_dict, f, indent=4)

accuracy of train_set/benign1.csv: 0.9632441070715142
accuracy of train_set/benign2.csv: 0.9627244340359095
accuracy of dataset_lite/mirai-benign.csv: 0.9968132106902295
accuracy of dataset_lite/BruteForce-Web.csv: 0.9854014598540146
accuracy of dataset_lite/BruteForce-XSS.csv: 0.9767441860465116
accuracy of dataset_lite/infiltration.csv: 0.8333333333333334
accuracy of dataset_lite/osscan.csv: 0.007331378299120235
accuracy of dataset_lite/SQL_Injection.csv: 0.9411764705882353
accuracy of dataset_lite/ssldosA10only.csv: 0.30434782608695654
accuracy of dataset_lite/mirai-attack.csv: 0.0031655714056739864


In [46]:
print(threshold)

0.0006182878278195858
